In [1]:
import pandas as pd
import numpy as np
import os

In [100]:
# This should point to base dir of regional_forecast git repo
# https://github.com/BayAreaMetro/regional_forecast
REGIONAL_FORECAST_GIT_DIR = r"\\mainmodel\MainModelShare\regional_forecast"

# The location of REMI outputs
REMI_OUTPUT_BOX_DIR       = os.path.join(os.environ["USERPROFILE"],
                            "Box\Modeling and Surveys\Regional Modeling\REMI\REMI files\PBA50 Blueprint\REMI output")

# e.g. Draft Forecast Base Case, Draft Forecast Upper Case, Forecast 03122020
FORECAST_DIR              = "Forecast 04132020"

# e.g. Blueprint Baseline, Blueprint Basic, Blueprint Crossing, Blueprint Plus Fix it First
FORECAST_VARIANT          = "Blueprint Plus Fix it First"

# Figure out corresponding UrbanSim Scenario number
# see https://github.com/BayAreaMetro/bayarea_urbansim/blob/zoning_mods_update/configs/scenarios.md
URBANSIM_VARIANT_MAPPING  = {"Blueprint Baseline"         :20,
                             "Blueprint Basic"            :21,
                             "Blueprint Plus Fix it First":22,
                             "Blueprint Crossing"         :23}
URBANSIM_VARIANT          = URBANSIM_VARIANT_MAPPING[FORECAST_VARIANT]

#step 1: output will into this workbook
employment_output_file    = os.path.join(REMI_OUTPUT_BOX_DIR, FORECAST_DIR, FORECAST_VARIANT, 
                                         "employment_controls_s{}.csv".format(URBANSIM_VARIANT))

population_output_file    = os.path.join(REMI_OUTPUT_BOX_DIR, FORECAST_DIR, FORECAST_VARIANT, 
                                         "population_controls_s{}.csv".format(URBANSIM_VARIANT))

# create readme
#readme = pd.DataFrame(data=[{"label":"Created by",
                             #"value":os.path.join(REGIONAL_FORECAST_GIT_DIR, "regionals post process",
                                                  #"Control Totals File for UrbanSim PBA50.ipynb")}])
#readme.to_excel(writer, sheet_name='readme')

# Convert Employment into 6 Categories for Travel Model

Where categories are: *RETEMPN*, *FPSEMPN*, *HEREMPN*, *AGREMPN*, *MWTEMPN*, *OTHEMPN*

See https://github.com/BayAreaMetro/modeling-website/wiki/TazData

In [101]:
# start by aggregating detailed industries into 6 industry categories
ind_6cat = pd.read_csv(os.path.join(REGIONAL_FORECAST_GIT_DIR, "regionals post process", "ind_6_cat.csv"))

#then read through all employment results--currently, I am separating the two scripts, ideally
#they could be combined
#we have different treatment for e-commerce employment, part of retail sector-see "MEMO E-Commerce in UrbanSim.docx" in the documentation folder
#we may need to decide what to do for the forecast
emp = pd.read_csv(os.path.join(REMI_OUTPUT_BOX_DIR, FORECAST_DIR, FORECAST_VARIANT, "emp by ind11_s{}.csv".format(URBANSIM_VARIANT)))

#merge with industry mapping
emp2 = pd.merge(emp, ind_6cat, on = 'ind', how = 'left' )
emp2010 = pd.read_csv(os.path.join(REMI_OUTPUT_BOX_DIR, FORECAST_DIR, FORECAST_VARIANT, "emp2010.csv"))


In [102]:
emp2

,ind,2015,2020,2025,2030,2035,2040,2045,2050,ind_6cat
0,Agriculture & Natural Resources,24894,23104,23321,26106,27039,28027,29021,29987,AGREMPN
1,"Arts, Recreation & Other Services",598143,507093,615796,696554,725950,763296,787649,813021,HEREMPN
2,Construction,207781,212482,249657,261216,273236,290458,296933,304811,OTHEMPN
3,Financial & Leasing,209891,212533,212738,235757,244942,257022,263053,268894,FPSEMPN
4,Government,466052,519733,480841,498460,502939,509267,503310,496729,OTHEMPN
5,Health & Educational Services,591292,618162,633314,740983,780092,823641,855531,889410,HEREMPN
6,Information,171880,205643,201243,245178,266979,294253,325335,360557,OTHEMPN
7,Manufacturing & Wholesale,472735,465777,414945,434519,428737,431378,434004,436478,MWTEMPN
8,Professional & Managerial Services,793726,837724,848437,977492,1033410,1092555,1153423,1214656,FPSEMPN
9,Retail,358070,360466,332291,379739,393971,419014,431883,444423,RETEMPN


In [103]:
emp_6cat           = emp2.groupby(['ind_6cat']).sum()
emp_6cat_transpose = emp_6cat.T.reset_index()
emp_controls       = emp_6cat_transpose.rename(columns={'index': 'year'})
emp_controls.year  = emp_controls.year.astype(int)
emp_controls       = emp_controls.reset_index(drop = True)

In [104]:
emp_controls_final = pd.concat([emp2010,emp_controls])

In [105]:
emp_controls_final =emp_controls_final.set_index('year').reset_index()

In [106]:
emp_controls_final

,year,AGREMPN,FPSEMPN,HEREMPN,MWTEMPN,OTHEMPN,RETEMPN
0,2010,27545,818157,867980,561534,546126,636935
1,2015,24894,1003617,1189435,583589,845713,358070
2,2020,23104,1050257,1125255,582468,937858,360466
3,2025,23321,1061175,1249110,549510,931741,332291
4,2030,26106,1213249,1437537,583220,1004854,379739
5,2035,27039,1278352,1506042,582962,1043154,393971
6,2040,28027,1349577,1586937,593651,1093978,419014
7,2045,29021,1416476,1643180,603713,1125578,431883
8,2050,29987,1483550,1702431,614258,1162097,444423


In [107]:
#write to the master excel file
emp_controls_final.to_csv(employment_output_file, header=True, index=False)

# Summarize Population by Age Group

Where age groups are *AGE0004*, *AGE0519*, *AGE2044*, *AGE4564*, *AGE65p*

See https://github.com/BayAreaMetro/modeling-website/wiki/TazData

In [108]:
#starts with aggregating pop by single age into 5 age groups
#I have created a relationship file, now read it
agegrp = pd.read_csv(os.path.join(REGIONAL_FORECAST_GIT_DIR, "regionals post process", "age_group.csv"))

#then read through all population results
pop = pd.read_csv(os.path.join(REMI_OUTPUT_BOX_DIR, FORECAST_DIR, FORECAST_VARIANT,"population_age_s{}.csv".format(URBANSIM_VARIANT)))

In [109]:
#summarize into age groups, needed two transposes
pop_indexed    = pop.set_index('Year')
pop2           = pop_indexed.T.reset_index()
pop2_merge     = pd.merge(pop2, agegrp, left_on = 'index', right_on = 'age', how = 'inner' )
pop2_sum       = pop2_merge.groupby(['age_group']).sum()
pop2_transpose = pop2_sum.T.reset_index()
pop_controls   = pop2_transpose.rename(columns={'index': 'Year'}).reset_index(drop = True)

In [110]:
pop_controls

age_group,Year,age0004,age0519,age2044,age4564,age65p
0,2010,447575,1321773,2581035,1931699,884968
1,2015,443193,1336227,2750266,2039829,1086042
2,2020,423822,1364797,2792002,2082033,1272452
3,2025,408531,1369155,2831248,2130958,1490333
4,2030,400880,1356484,2855026,2235717,1706808
5,2035,416470,1366483,2915326,2413851,1883387
6,2040,448143,1402936,3067506,2552610,2028175
7,2045,467546,1458945,3253998,2603670,2169704
8,2050,474979,1522143,3342822,2666121,2358347


In [111]:
pop_controls.to_csv(population_output_file, header=True, index=False)

# additional step for calculating share of 4 age groups using the transposed pop2
agegrp_v2 = pd.read_csv('C:/Users/blu/Box/RTP SCS 2021 Modeling/control_totals/deliverables/4 age groups.csv')
pop2_4ag = pd.merge(pop2, agegrp_v2, left_on = 'index', right_on = 'age', how = 'inner' )
pop2_4agsum = pop2_4ag.groupby(['age_group']).sum()
pop2_4agtranspose = pop2_4agsum.T.reset_index()
pop_4ag = pop2_4agtranspose.rename(columns={'index': 'year'}).reset_index(drop = True)

pop_4ag.head()

pop_4ag['total'] = pop_4ag['age0019'] + pop_4ag['age2034'] + pop_4ag['age3564'] + pop_4ag['age65p']  
pop_4ag['shra1'] = pop_4ag['age0019']/pop_4ag['total']
pop_4ag['shra2'] = pop_4ag['age2034']/pop_4ag['total']
pop_4ag['shra3'] = pop_4ag['age3564']/pop_4ag['total']
pop_4ag['shra4'] = pop_4ag['age65p']/pop_4ag['total']
pop_4ag.head()

#this is the tricky part to add share of children
#first add column, then calculate value
pop_4ag['shry'] = ''
#2010 children share is given
pop_4ag.shry.iloc[0]= 0.33350388
rownum = len(pop_4ag.index)
for i in range(1, rownum):
    pop_4ag.shry.iloc[i] = pop_4ag.shry.iloc[i-1] * ( pop_4ag.shra1.iloc[i]/pop_4ag.shra1.iloc[i-1])
#then add no children share
pop_4ag['shrn'] = 1 - pop_4ag['shry']

pop_4ag.head()

# Transfer Household Results Directly into Output Workbook

In [12]:
#read through household results
hh_file = os.path.join(REMI_OUTPUT_BOX_DIR, FORECAST_DIR, FORECAST_VARIANT,
                       "household_controls_s{}.csv".format(URBANSIM_VARIANT))
hh = pd.read_csv(hh_file)

In [13]:
hh

,year,q1_households,q2_households,q3_households,q4_households
0,2015,688276,657079,584177,753096
1,2020,686227,627931,612707,867514
2,2025,782659,670828,646543,967700
3,2030,887095,719522,675853,1076883
4,2035,954607,744042,694685,1148066
5,2040,1012602,768913,723641,1217992
6,2045,1063297,789688,756240,1282931
7,2050,1110271,803318,786704,1347164


In [14]:
hh.to_excel(writer, sheet_name='hh_controls')

In [15]:
pop_controls['totpop'] = pop_controls['age0004'] + \
                         pop_controls['age0519'] + \
                         pop_controls['age2044'] + \
                         pop_controls['age4564'] + \
                         pop_controls['age65p']
hh['tothh']            = hh['q1_households'] + \
                         hh['q2_households'] + \
                         hh['q3_households'] + \
                         hh['q4_households']
emp_controls['totemp'] = emp_controls['AGREMPN'] + \
                         emp_controls['MWTEMPN'] + \
                         emp_controls['RETEMPN'] + \
                         emp_controls['FPSEMPN'] + \
                         emp_controls['HEREMPN'] + \
                         emp_controls['OTHEMPN']

In [16]:
regional_controls = pd.merge(pop_controls, emp_controls)
regional_controls = pd.merge(regional_controls, hh)
regional_controls = regional_controls[["year","totpop","tothh","totemp"]]

In [17]:
regional_controls

,year,totpop,tothh,totemp
0,2015,7655557,2682628,4007439
1,2020,7955156,2794379,4445568
2,2025,8397502,3067730,4566346
3,2030,8712598,3359353,4661859
4,2035,9060735,3541400,4835443
5,2040,9500663,3723148,5053270
6,2045,9917230,3892156,5227191
7,2050,10308095,4047457,5409015


In [18]:
regional_controls.to_excel(writer, sheet_name='regional_controls')

In [22]:
writer.save()

## sum and write to master excel file
hh_sum = hh.groupby(['year','inc_cat_mtc_int']).sum().reset_index()
hh_controls = hh_sum.pivot(index='year', columns='inc_cat_mtc_int', values='total').reset_index()
hh_controls.to_excel(writer, sheet_name='hh_controls')

hh_q = hh_controls
hh_q.head()

#calculate share by inc quarter
hh_q['total'] = hh_q['q1'] + hh_q['q2'] + hh_q['q3'] + hh_q['q4']  
hh_q['shrq1'] = hh_q['q1']/hh_q['total']
hh_q['shrq2'] = hh_q['q2']/hh_q['total']
hh_q['shrq3'] = hh_q['q3']/hh_q['total']
hh_q['shrq4'] = hh_q['q4']/hh_q['total']

hh_q.head()

#calculate share by number of workers
hh_sumbyworker = hh.groupby(['year','hhworkers']).sum().reset_index()
hh_w = hh_sumbyworker.pivot(index='year', columns='hhworkers', values='total').reset_index()
hh_w['total'] = hh_w['w0'] + hh_w['w1'] + hh_w['w2'] + hh_w['w3'] 
hh_w['shrw0'] = hh_w['w0']/hh_w['total']
hh_w['shrw1'] = hh_w['w1']/hh_w['total']
hh_w['shrw2'] = hh_w['w2']/hh_w['total']
hh_w['shrw3'] = hh_w['w3']/hh_w['total']
hh_w.head()

#calculate share by hh size
hh_sumbysize = hh.groupby(['year','hhsize']).sum().reset_index()
hh_s = hh_sumbysize.pivot(index='year', columns='hhsize', values='total').reset_index()
hh_s['total'] = hh_s['s1'] + hh_s['s2'] + hh_s['s3'] + hh_s['s4']  
hh_s['shrs1'] = hh_s['s1']/hh_s['total']
hh_s['shrs2'] = hh_s['s2']/hh_s['total']
hh_s['shrs3'] = hh_s['s3']/hh_s['total']
hh_s['shrs4'] = hh_s['s4']/hh_s['total']
hh_s.head()

#Step 5 add all hh and pop share into a dataframe
reg_dem_fcast = pd.DataFrame()
b_slice1 = hh_s[['year','shrs1','shrs2','shrs3','shrs4']].copy()
b_slice2 = hh_q[['year','shrq1','shrq2','shrq3','shrq4']].copy()
b_slice3 = hh_w[['year','shrw0','shrw1','shrw2','shrw3']].copy()
b_slice4 = 4ag[['year','shra1','shra2','shra3','shra4','shrn','shry']].copy()
reg_dem_fcast = reg_dem_fcast.append(b_slice1)
reg_dem_fcast1 = pd.merge(reg_dem_fcast,b_slice2, on = 'year', how = 'left')
reg_dem_fcast2 = pd.merge(reg_dem_fcast1,b_slice3, on = 'year', how = 'left')
reg_dem_fcast3 = pd.merge(reg_dem_fcast2,b_slice4, on = 'year', how = 'left')

reg_dem_fcast3.head()

reg_dem_fcast3.to_excel(writer, sheet_name='reg_dem_fcast')

# last step, putting together regional control master file
#vacany rate
vr= pd.read_csv('C:/Users/blu/Box/RTP SCS 2021 Modeling/control_totals/deliverables/vacancy rate.csv')

#pop
pop_controls['totpop'] = pop_controls['age0004'] + pop_controls['age0519']+ pop_controls['age2044']+pop_controls['age4564'] + pop_controls['age65p']

#employed residents
#first set up a constant for rate of employed residents -- update if needed.
empres_r = 0.98
#then put everything together
emp_controls.year = emp_controls.year.astype(int)
emp_controls['empres'] = empres_r * (emp_controls['AGREMPN'] +emp_controls['MWTEMPN'] + emp_controls['RETEMPN'] + emp_controls['FPSEMPN'] + emp_controls['HEREMPN'] + emp_controls['OTHEMPN'])
empres = emp_controls[['year','empres']]

#five age groups
pop_5ag = pop_controls[['year','age0004','age0519','age2044','age4564','age65p']]

regional_controls = pop_controls[['year','totpop']] 
regional_controls1 = pd.merge(regional_controls, empres, on = 'year', how = 'left')
regional_controls2 = pd.merge(regional_controls1, pop_5ag, on = 'year', how = 'left')
regional_controls3 = pd.merge(regional_controls2, vr, on = 'year', how = 'left')

#add static median age -- update if needed
medianAge = [37.7,38.5,38.9,39.6,40.2,40.9,42.7,42.2,42.9]
regional_controls3.insert(loc=3, column='medianage', value=medianAge)

regional_controls3.head()

regional_controls3.to_excel(writer, sheet_name='regional_controls')

writer.save()